In [1]:

import numpy as np
import torch
from scipy.integrate import odeint
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from torch import nn, autograd


In [ ]:
# Define a function to solve the Duffing equation numerically
def solve_duffing(d, a, b, gamma, w, x):
    def duffing(y, t):
        y0, y1 = y
        dydt = [y1, -d * y1 - a * y0 - b * y0**3 + gamma * np.cos(w * t)]
        return dydt

    y0 = [0, 0]  
    sol = odeint(duffing, y0, x.view(-1).numpy())  # Convert x to a one-dimensional numpy array
    y = torch.tensor(sol[:, 0])
    return y


In [2]:
# Define a class to create a fully connected neural network
class FCN(nn.Module):
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
                # Assuming your linear layer is named `layer`
        input_size = self.fce.in_features
        print("Input size of the linear layer:", input_size)
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

In [3]:
# Defining the Duffing class
class Duffing:
    def __init__(self, d, a, b, gamma, w, initial_conditions=(0, 0)):
        """
        Initialize the Duffing equation parameters.

        :param d: Damping coefficient.
        :param a: Linear stiffness coefficient.
        :param b: Nonlinear stiffness coefficient.
        :param gamma: Forcing amplitude.
        :param w: Frequency of the forcing.
        :param initial_conditions: Tuple of initial conditions (position, velocity).
        """
        self.d = d
        self.a = a
        self.b = b
        self.gamma = gamma
        self.w = w
        self.initial_conditions = initial_conditions

    def solve(self, time_steps):
        """
        Solve the Duffing equation numerically.

        :param time_steps: Array of time steps for the solution.
        :return: Solution of the Duffing equation at the given time steps.
        """
        def duffing_equation(y, t):
            y0, y1 = y
            dydt = [y1, -self.d * y1 - self.a * y0 - self.b * y0**3 + self.gamma * np.cos(self.w * t)]
            return dydt

        sol = odeint(duffing_equation, self.initial_conditions, time_steps)
        y = torch.tensor(sol[:, 0])
        return y


# Example usage of the Duffing class
# Example parameters for the Duffing equation
duffing_parameters = {
    "d": 2,
    "a": 10.8,
    "b": 5.0,
    "gamma": 50,
    "w": 10
}

# Creating an instance of the Duffing class
duffing_oscillator = Duffing(**duffing_parameters)

# Solving the Duffing equation for a range of time steps
time_steps = np.linspace(0, 10, 1000)  # Time steps from 0 to 10 with 1000 points
solution = duffing_oscillator.solve(time_steps)

In [4]:

class TimeStepManager:
    def __init__(self, duffing_instance, n_physics, n_data):
        """
        Initialize the time step manager with a Duffing instance.

        :param duffing_instance: An instance of the Duffing class.
        :param n_physics: Number of time steps for physics-based training.
        :param n_data: Number of time steps for data-based training.
        """
        self.duffing_instance = duffing_instance
        self.n_physics = n_physics
        self.n_data = n_data
        self.x_physics = None
        self.x_data = None

    def generate_physics_time_steps(self):
        """Generate time steps for physics-based training, covering 30% of the Duffing time frame."""
        start, end = self.duffing_instance.time_horizon
        time_frame = (end - start) * 0.3
        self.x_physics = torch.linspace(start, start + time_frame, self.n_physics).view(-1, 1)

    def generate_data_time_steps(self):
        """Generate time steps for data-based training, covering 30% of the Duffing time frame."""
        start, end = self.duffing_instance.time_horizon
        time_frame = (end - start) * 0.3
        self.x_data = torch.linspace(start, start + time_frame, self.n_data).view(-1, 1)

    def get_physics_time_steps(self):
        """Return the physics time steps."""
        return self.x_physics

    def get_data_time_steps(self):
        """Return the data time steps."""
        return self.x_data

# Using the updated TimeStepManager with the Duffing instance
time_step_manager = TimeStepManager(duffing_oscillator, 30, 500)
time_step_manager.generate_physics_time_steps()
time_step_manager.generate_data_time_steps()
# Example usage
time_step_manager = TimeStepManager(0, 1, 30, 500)
time_step_manager.generate_physics_time_steps()
time_step_manager.generate_data_time_steps()

# Accessing the generated time steps
x_physics = time_step_manager.get_physics_time_steps()
x_data = time_step_manager.get_data_time_steps()
y_data = y[x_data]

AttributeError: 'Duffing' object has no attribute 'time_horizon'

In [ ]:


  
# Set up the parameters and the training data for the model
d = 2
a = 0.0
b = 0.0 
gamma = 30
w = 20
x = torch.linspace(0, 1, 500).view(-1,1) 
y = solve_duffing(d, a, b, gamma, w, x).view(-1,1)
y_data = y[0:200:20]
x_data = x[0:200:20]
displacement_data = []
velocity_data = []   

In [ ]:


def plot_duffing_oscillator(d, a, b, gamma, w, x, y):
    plt.figure(figsize=(10, 6))
    plt.plot(x.view(-1).numpy(), y.view(-1).numpy(), label='Duffing Oscillation', color="grey", linewidth=2)
    plt.xlabel('Time')
    plt.ylabel('Position')
    plt.title('Duffing Oscillator')
    plt.legend()
    plt.grid(True)
    plt.show()  # Display the plot immediately

# Plot the Duffing oscillator data
plot_duffing_oscillator(d, a, b, gamma, w, x, y)
plt.show()

In [ ]:





# Set up the physics loss training locations
#x_physics = torch.linspace(0, 1, 30).view(-1, 1).requires_grad_(True)

# Set up the random seed and initialize the model and optimizer
torch.manual_seed(123)
model = FCN(1,1,32,12)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Set up the boundary conditions
X_BOUNDARY = 0.0  # bondary condition coordinate
F_BOUNDARY = 0.0  # boundary condition value

# Train the model for 60000 steps
for i in range(6000):
    optimizer.zero_grad()
   
    yh = model(x_data)
    # Compute the data loss by comparing the model output with the training data
   
    loss1 = torch.mean((yh - y_data)**2)
    
   
    # Compute the physics loss by enforcing the differential equation
    yhp = model(x_physics)
    dy_pred = autograd.grad(yhp, x_physics, torch.ones_like(yhp), create_graph=True)[0]
    d2y_pred = autograd.grad(dy_pred, x_physics, torch.ones_like(dy_pred), create_graph=True)[0]

    physics = d2y_pred + d * dy_pred + a * yhp + b * torch.pow(yhp, 3) - gamma * torch.cos(w * x_physics)
    loss_physics = (1e-4) * torch.mean(physics**2)

    # Compute the boundary loss by enforcing the boundary conditions
    x_boundary = torch.tensor([X_BOUNDARY]).view(-1, 1).requires_grad_(True)
    f_boundary = torch.tensor([F_BOUNDARY]).view(-1, 1)
    yh_boundary = model(x_boundary)
    boundary = yh_boundary - f_boundary
    loss_boundary = torch.mean(boundary**2)

    # Compute the total loss as the sum of the data loss, the physics loss, and the boundary loss
    total_loss = loss1 + loss_physics + loss_boundary

    # Update the model parameters using backpropagation and gradient descent
    total_loss.backward()
    optimizer.step()
    # Initialize the self-feeding process
    x_self_feed = x[0].reshape(1, -1)  # Starting with the first time step
    predictions = []  # To store the model's predictions
    n_steps = 100  # Define the number of steps for the self-feeding process

    for _ in range(n_steps):
        # Make a prediction using the current input
        with torch.no_grad():
            y_pred = model(x_self_feed)

        # Store the prediction
        predictions.append(y_pred.item())

        # Use the prediction as the next input
        x_self_feed = y_pred

# Convert predictions to a tensor for analysis
predictions_tensor = torch.tensor(predictions).view(-1, 1)
# Ground-truth data for comparison (assuming y contains the numerical solution)
ground_truth = y[:n_steps]

# Calculate the difference between predictions and ground-truth
difference = torch.abs(predictions_tensor - ground_truth)

# Analyzing stability - Criteria could be based on the magnitude of the difference
# For example, finding the step at which the difference exceeds a certain threshold
threshold = 0.1  # Define a suitable threshold
unstable_step = torch.where(difference > threshold)[0].min().item()

# Output the step at which the predictions become unstable
print(f'Predictions become unstable at step: {unstable_step}')


     
    # Plot the results every 150 steps
if (i+1) % 1000 == 0: 
 # Print the loss value after each step
    print("Loss at Step", i+1, ":", total_loss.item())
    yh = model(x).detach().numpy()
    plt.figure(figsize=(10, 5))
    plt.plot(x.numpy(), y.numpy(), label='Ground Truth')
    plt.plot(x.numpy(), yh, label='Neural Network Output')
    plt.scatter(x_data.numpy(), y_data.numpy(), color='red', label='Training points')
    plt.legend()
    plt.show()



In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(predictions_tensor[:unstable_step], label='Predictions', color='blue')
plt.plot(ground_truth[:unstable_step], label='Ground Truth', color='green')
plt.axhline(y=threshold, color='red', linestyle='--', label='Threshold')
plt.xlabel('Time Steps')
plt.ylabel('Values')
plt.title(f'Prediction and Ground Truth up to Unstable Step {unstable_step}')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(predictions_tensor, label='Predictions', color='blue')  # Plot the entire predictions
plt.plot(ground_truth, label='Ground Truth', color='green')  # Plot the entire ground truth
plt.axhline(y=threshold, color='red', linestyle='--', label='Threshold')  # Threshold line
plt.xlabel('Time Steps')
plt.ylabel('Values')
plt.title('Prediction and Ground Truth over Every Step')
plt.legend()
plt.grid(True)
plt.show()